In [209]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [210]:
def get_values(elm, price_type):
    price_all = []
    elm_check_digit = elm.split(' ')
    for i in elm_check_digit:
        e = i.replace('\xa0', ' ')
        elm_digit = e.split(' ')
        elm = []
        for i2 in elm_digit:
            if i2.isdigit():
                elm.append(i2)
        if(elm):
            price_all.append(" ".join(elm))
    price_temp = " ".join(price_all)
    return price_temp + " " + price_type

In [211]:
def calculate_price(price):
    price_type = price.split(' ')[-1]
    if price.find('-') != -1:
        price = price.split('-')
        elms_compensation = []
        for elm in price:
            price_all.append(get_values(elm, price_type))
    elif price.find('от') != -1:
        price_all.append('None')
        price_all.append(get_values(price, price_type))
    elif price.find('до') != -1:
        price_all.append(get_values(price, price_type))
        price_all.append('None')
    return price_all

In [218]:
main_url = 'https://hh.ru'
base_url = main_url + '/search/vacancy?area=113&text=Data+science&page='
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.96 YaBrowser/20.4.0.3443 Yowser/2.5 Yptp/1.23 Safari/537.36'
headers = {'accept': '*/*', 'user-agent': user_agent}    
is_true = True

jobs =[]
job_title = []
job_price_min = []
job_price_max = []
job_href = []
job_site = []

while is_true:
    request = requests.get(base_url, headers = headers)
    soup = BeautifulSoup(request.content, 'lxml')
    divs = soup.find_all('div', {'data-qa': 'vacancy-serp__vacancy', 'class': 'vacancy-serp-item'})
    for div in divs:
        price_all = []
        title = div.find('a', attrs = {'data-qa': 'vacancy-serp__vacancy-title'}).text
        div_price = div.find('span', attrs={'data-qa': 'vacancy-serp__vacancy-compensation'})
        if div_price == None:
            price_all.append('None')
            price_all.append('None')
        else:
            price = div.find('span', attrs={'data-qa': 'vacancy-serp__vacancy-compensation'}).text
            price_all = calculate_price(price)

        href = div.find('a', attrs = {'data-qa': 'vacancy-serp__vacancy-title'})['href'].split('?')[0]
        all_txt = [title, price_all[0], price_all[1], href, 'hh.ru']
        jobs.append(all_txt)

        job_title.append(title)
        job_price_min.append(price_all[0])
        job_price_max.append(price_all[1])
        job_href.append(href)
        job_site.append('hh.ru')
    page_next = soup.find('a', {"data-qa": "pager-next", "class": "bloko-button HH-Pager-Controls-Next HH-Pager-Control"})
    if page_next != None:
        base_url = main_url + page_next['href']
        is_true = True
    else:
        is_true = False
    time.sleep(1)

In [219]:
all_jobs = {
    'Title': job_title,
    'Price Min': job_price_min,
    'Price Max' : job_price_max,
    'Link' : job_href,
    'hh.ru' : job_site
}

df = pd.DataFrame(all_jobs, columns = ['Title', 'Price Min', 'Price Max', 'Link', 'hh.ru'])

In [220]:
df

,Title,Price Min,Price Max,Link,hh.ru
0,Python Data Scientist,None,150 000 руб.,https://krasnodar.hh.ru/vacancy/36916080,hh.ru
1,Data Engineer - Sales Home,None,220 000 руб.,https://krasnodar.hh.ru/vacancy/37036573,hh.ru
2,Руководитель локальной группы Data Science в г...,None,None,https://krasnodar.hh.ru/vacancy/36672688,hh.ru
3,Data Scientist,None,None,https://krasnodar.hh.ru/vacancy/36023475,hh.ru
4,Аналитик данных / BI аналитик,None,None,https://krasnodar.hh.ru/vacancy/37356801,hh.ru
...,...,...,...,...,...
525,Compilers and Run-time Developer,None,None,https://krasnodar.hh.ru/vacancy/36928101,hh.ru
526,Android Developer,None,None,https://krasnodar.hh.ru/vacancy/36924028,hh.ru
527,Senior Android developer,None,None,https://krasnodar.hh.ru/vacancy/36634291,hh.ru
528,Разработчик навигационных алгоритмов,None,None,https://krasnodar.hh.ru/vacancy/37082078,hh.ru


In [223]:
df.to_csv('hh.ru-data science utf-8.csv', header=True, index=False, encoding='utf-8')

In [238]:
with open('ready_hh.txt', 'w+', encoding='utf-8') as f:
    for line in jobs:
        f.write(line[0]+'\t'+line[1]+'\t'+line[2]+'\t'+line[3]+'\thh.ru')